In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_data=pd.read_excel(r"D:\数模\第四次模拟题——在线零售商的订单履行\附件一.xlsx")
raw_data.head()

,订单,订单中的SKU,订单对应的顾客位置坐标
0,O00001,"[77,63,94,1,36,10,70,86,14,67]","[71.0270441124638,33.8644796735834]"
1,O00002,"[36,25,24,33,92,76]","[55.4853668373032,76.1047715540322]"
2,O00003,"[74,23,26,89]","[47.9812275454335,35.6245047959696]"
3,O00004,"[67,20]","[46.5971290026430,42.0066352580043]"
4,O00005,"[82,34,61,69,63,87,28,75]","[54.4360971527062,76.6912521649171]"


In [3]:
SKU1=raw_data['订单中的SKU']
SKU_in_order=[]
# 原有数据类型为'[]'型，先进行转化,最终要得到['','']的形式
for order in SKU1:
    if type(order)==int:
        SKU_in_order.append(str(order))
    else:
        order=order[1:-1].split(",")
        SKU_in_order.append(order)

print(len(SKU_in_order))

50000


In [4]:
cnt=np.zeros(101)
for i in SKU_in_order:
    if type(i)==list:
        for j in i:
            cnt[int(j)]+=1
    else :
        cnt[int(i)]+=1
 
# print(cnt)
# print(cnt.sum())

In [5]:
key_val=[]
for i in range(1,101):
    key_val.append([i,cnt[i]])

key_val=pd.DataFrame(key_val)
d=key_val.sort_values(1,ascending=False)
SKU_sort=d.iloc[:,0].values

In [6]:
# 构造共现矩阵
ros=np.zeros((50000,100))
for i in range(len(SKU_in_order)):

    if type(SKU_in_order[i])==str:
        ros[i][int(SKU_in_order[i])-1]=1
    else:
        for j in SKU_in_order[i]:
            ros[i][int(j)-1]=1


# for i in range(100):
#     print(ros[i,:].sum())
R=np.dot(ros.T,ros)
print(R.trace())
print(R.shape)

275269.0
(100, 100)


In [7]:
pos_w1=[0.516205320509211,83.7701245337600]

pos_w2=[30.0817907887535,1.17493342829637]

pos_w3=[58.8349268443702,89.8949407626723]

In [8]:
pos_order=raw_data['订单对应的顾客位置坐标']
pos_o=[]
for i in pos_order:
    i=i[1:-1].split(",")
    pos_o.append([float(t) for t in i])

In [9]:
import math
def dis(order_pos):
    dis1=math.sqrt((order_pos[0]-pos_w1[0])**2+(order_pos[1]-pos_w1[1])**2)
    dis2=math.sqrt((order_pos[0]-pos_w2[0])**2+(order_pos[1]-pos_w2[1])**2)
    dis3=math.sqrt((order_pos[0]-pos_w3[0])**2+(order_pos[1]-pos_w3[1])**2)
    return [dis1,dis2,dis3]

In [10]:
# 贪婪热销算法   #仓库大小60
# SKU_sort 为排好序的数组
# x个货品在三个仓库都有，y个货品在两个仓库里有，z个货品只有一个仓库有
# x+y+z=100
# 3*x+2*y+z=3*60
#  X=Z-20
#  y=80-2x
bestcost=9999999
bestW1=[]
bestW2=[]
bestW3=[]
bestx=0
c1=0
c2=0
for x in range(1,60):
    x=12
    # 准备仓库
    W1=[]
    W2=[]
    W3=[]

    # x
    for i in range(x):
        W1.append(SKU_sort[i]-1)
        W2.append(SKU_sort[i]-1)
        W3.append(SKU_sort[i]-1)

    # print(W1)
    # print(len(W1))


    # y
    if x<80-x:
        for i in range(x,80-x):

            item=SKU_sort[i]-1

            sum1=0
            if len(W1)<60:
                for j in W1:
                    sum1+=R[item][j]
            sum1=sum1/len(W1)
            #else :sum1=99999

            sum2=0
            if len(W2)<60:
                for j in W2:
                    sum2+=R[item][j]
            sum2=sum2/len(W2)
            #else :sum2=99999

            sum3=0
            if len(W3)<60:
                for j in W3:
                    sum3+=R[item][j]
            sum3=sum3/len(W3)
            #else :sum3=99999
            
            if sum1!=0 and sum2!=0 and sum3!=0:
                if min(sum1,sum2,sum3)==sum1:
                    W2.append(item)
                    W3.append(item)
                elif min(sum1,sum2,sum3)==sum2:
                    W1.append(item)
                    W3.append(item)
                elif min(sum1,sum2,sum3)==sum3:
                    W1.append(item)
                    W2.append(item)
            
            if sum1==0 or sum2==0 or sum3==0:
                if sum1!=0:
                    W1.append(item)
                if sum2!=0:
                    W2.append(item)
                if sum3!=0:
                    W3.append(item)
        # print(len(W1))
        # print(len(W2))

    # z   
    for i in range(80-x,100):

        item=SKU_sort[i]-1

        sum1=0
        if len(W1)<60:
            for j in W1:
                sum1+=R[item][j]
        sum1=sum1/len(W1)

        sum2=0
        if len(W2)<60:
            for j in W2:
                sum2+=R[item][j]
        sum2=sum2/len(W2)

        sum3=0
        if len(W3)<60:
            for j in W3:
                sum3+=R[item][j]
        sum3=sum3/len(W3)
            
        if max(sum1,sum2,sum3)==sum1 and sum1!=0:
                    W1.append(item)
        elif max(sum1,sum2,sum3)==sum2 and sum2!=0:
                    W2.append(item)
        elif max(sum1,sum2,sum3)==sum3 and sum3!=0:
                    W3.append(item)

    # 序号从 0-99变为1—100
    for i in range(len(W1)):
        W1[i]=W1[i]+1
    for i in range(len(W2)):
        W2[i]=W2[i]+1
    for i in range(len(W3)):
        W3[i]=W3[i]+1
    W_S1=set(W1)
    W_S2=set(W2)
    W_S3=set(W3)
    cost=0  # 所有订单在一个x下的花费
    cm_1=0
    cm_2=0
    
    for sk in range(len(SKU_in_order)):
        scost=9999 # 每个订单的花费
        o_sk=SKU_in_order[sk]
        if type(o_sk)==str:
            ii=[int(o_sk)]
        else:
            ii=[int(t) for t in o_sk]

        o_dis=dis(pos_o[sk])
        p_dis=np.zeros(3)
        #计算各自距离
        for j in range(3):
            if o_dis[j]>5:
                p_dis[j]=0.2*(o_dis[j]-5)+3
            else :
                p_dis[j]=3
        # 一个仓库就可以满足订单
        if set(ii).issubset(W_S1) or set(ii).issubset(W_S2) or set(ii).issubset(W_S3):
            cm_1+=1
            if set(ii).issubset(W_S1):
                scost=min(scost,p_dis[0])
            if set(ii).issubset(W_S2):
                scost=min(scost,p_dis[1])
            if set(ii).issubset(W_S3):
                scost=min(scost,p_dis[2])
        # 两个仓库才能满足订单
        elif set(ii).issubset(W_S1 | W_S2) or set(ii).issubset(W_S2 | W_S3) or set(ii).issubset(W_S3|W_S1):
            cm_2+=1
            if set(ii).issubset(W_S1|W_S2):
                scost=min(scost,p_dis[0]+p_dis[1])
            if set(ii).issubset(W_S2|W_S3):
                scost=min(scost,p_dis[1]+p_dis[2])
            if set(ii).issubset(W_S3|W_S1):
                scost=min(scost,p_dis[2]+p_dis[0])
        # 三个仓库才能满足订单
        else:
            scost=p_dis.sum()
        cost+=scost
    #计算对某一x，所有订单产生的花费
    if cost<bestcost:
            bestcost=cost
            bestW1=W1
            bestW2=W2
            bestW3=W3
            c1=cm_1
            c2=cm_2
            bestx=x

In [11]:
print(bestcost)
print(len(bestW1))
print(len(bestW2))
print(len(bestW3))
print(c1)
print(c2)
print(c1+c2*2)  #84503
sb1=set(bestW1)
sb2=set(bestW2)
sb3=set(bestW3)

print(len(sb1|sb2|sb3))
print(bestx)
print(bestW1)
print(bestW2)
print(bestW3)

1096904.2212352317
60
60
60
16372
28866
74104
100
12
[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 61, 71, 30, 36, 41, 82, 38, 39, 78, 56, 83, 45, 50, 87, 3, 15, 81, 23, 28, 65, 92, 60, 37, 35, 74, 58, 53, 90, 75, 11, 72, 85, 8, 14, 44, 80, 46, 19, 21, 96, 33, 6, 26, 47, 2, 95, 12, 9]
[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 5, 30, 36, 70, 77, 89, 83, 25, 100, 87, 3, 15, 93, 16, 23, 28, 17, 60, 13, 74, 27, 75, 11, 68, 85, 63, 86, 8, 14, 44, 4, 22, 73, 20, 24, 64, 18, 32, 97, 69, 34, 49]
[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 41, 82, 70, 38, 77, 39, 89, 78, 56, 25, 45, 50, 100, 93, 16, 81, 65, 92, 17, 37, 13, 35, 58, 27, 53, 90, 68, 72, 63, 86, 80, 84, 42, 29, 10, 88, 1, 57, 7]


In [12]:
# 贪婪热销算法   #仓库大小80
# SKU_sort 为排好序的数组
# x个货品在三个仓库都有，y个货品在两个仓库里有，z个货品只有一个仓库有
# x+y+z=100
# 3*x+2*y+z=3*80=240
#  X=Z+40
#  y=140-2x
bestcost=9999999
bestW1=[]
bestW2=[]
bestW3=[]
bestx=0
c1=0
c2=0
for x in range(40,80):
    
    # 准备仓库
    W1=[]
    W2=[]
    W3=[]

    # x
    for i in range(x):
        W1.append(SKU_sort[i]-1)
        W2.append(SKU_sort[i]-1)
        W3.append(SKU_sort[i]-1)

    # print(W1)
    # print(len(W1))


    # y
    if x<140-x:
        for i in range(x,min(140-x,100)):

            item=SKU_sort[i]-1

            sum1=0
            if len(W1)<80:
                for j in W1:
                    sum1+=R[item][j]
            sum1=sum1/len(W1)
            #else :sum1=99999

            sum2=0
            if len(W2)<80:
                for j in W2:
                    sum2+=R[item][j]
            sum2=sum2/len(W2)
            #else :sum2=99999

            sum3=0
            if len(W3)<80:
                for j in W3:
                    sum3+=R[item][j]
            sum3=sum3/len(W3)
            #else :sum3=99999
            
            if sum1!=0 and sum2!=0 and sum3!=0:
                if min(sum1,sum2,sum3)==sum1:
                    W2.append(item)
                    W3.append(item)
                elif min(sum1,sum2,sum3)==sum2:
                    W1.append(item)
                    W3.append(item)
                elif min(sum1,sum2,sum3)==sum3:
                    W1.append(item)
                    W2.append(item)
            
            if sum1==0 or sum2==0 or sum3==0:
                if sum1!=0:
                    W1.append(item)
                if sum2!=0:
                    W2.append(item)
                if sum3!=0:
                    W3.append(item)
        # print(len(W1))
        # print(len(W2))

    # z   
    if 140-x<100:
        for i in range(140-x,100):

            item=SKU_sort[i]-1

            sum1=0
            if len(W1)<80:
                for j in W1:
                    sum1+=R[item][j]
            sum1=sum1/len(W1)

            sum2=0
            if len(W2)<80:
                for j in W2:
                    sum2+=R[item][j]
            sum2=sum2/len(W2)

            sum3=0
            if len(W3)<80:
                for j in W3:
                    sum3+=R[item][j]
            sum3=sum3/len(W3)
                
            if max(sum1,sum2,sum3)==sum1 and sum1!=0:
                W1.append(item)
            elif max(sum1,sum2,sum3)==sum2 and sum2!=0:
                W2.append(item)
            elif max(sum1,sum2,sum3)==sum3 and sum3!=0:
                W3.append(item)

    # 序号从 0-99变为1—100
    for i in range(len(W1)):
        W1[i]=W1[i]+1
    for i in range(len(W2)):
        W2[i]=W2[i]+1
    for i in range(len(W3)):
        W3[i]=W3[i]+1
    W_S1=set(W1)
    W_S2=set(W2)
    W_S3=set(W3)
    cost=0  # 所有订单在一个x下的花费
    cm_1=0
    cm_2=0
    
    for sk in range(len(SKU_in_order)):
        scost=9999 # 每个订单的花费
        o_sk=SKU_in_order[sk]
        if type(o_sk)==str:
            ii=[int(o_sk)]
        else:
            ii=[int(t) for t in o_sk]

        o_dis=dis(pos_o[sk])
        p_dis=np.zeros(3)
        #计算各自距离
        for j in range(3):
            if o_dis[j]>5:
                p_dis[j]=0.2*(o_dis[j]-5)+3
            else :
                p_dis[j]=3
        # 一个仓库就可以满足订单
        if set(ii).issubset(W_S1) or set(ii).issubset(W_S2) or set(ii).issubset(W_S3):
            cm_1+=1
            if set(ii).issubset(W_S1):
                scost=min(scost,p_dis[0])
            if set(ii).issubset(W_S2):
                scost=min(scost,p_dis[1])
            if set(ii).issubset(W_S3):
                scost=min(scost,p_dis[2])
        # 两个仓库才能满足订单
        elif set(ii).issubset(W_S1 | W_S2) or set(ii).issubset(W_S2 | W_S3) or set(ii).issubset(W_S3|W_S1):
            cm_2+=1
            if set(ii).issubset(W_S1|W_S2):
                scost=min(scost,p_dis[0]+p_dis[1])
            if set(ii).issubset(W_S2|W_S3):
                scost=min(scost,p_dis[1]+p_dis[2])
            if set(ii).issubset(W_S3|W_S1):
                scost=min(scost,p_dis[2]+p_dis[0])
        # 三个仓库才能满足订单
        else:
            scost=p_dis.sum()
        cost+=scost
    #计算对某一x，所有订单产生的花费
    if cost<bestcost:
            bestcost=cost
            bestW1=W1
            bestW2=W2
            bestW3=W3
            c1=cm_1
            c2=cm_2
            bestx=x

In [13]:
print(bestcost)
print(len(bestW1))
print(len(bestW2))
print(len(bestW3))
print(c1)
print(c2)
print(c1+c2*2) 
sb1=set(bestW1)
sb2=set(bestW2)
sb3=set(bestW3)

print(len(sb1|sb2|sb3))
print(bestx)
print(bestW1)
print(bestW2)
print(bestW3)

750902.1190334755
80
80
80
34064
15936
65936
100
42
[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 30, 36, 41, 82, 70, 38, 77, 39, 89, 78, 56, 83, 25, 45, 50, 100, 87, 3, 15, 93, 16, 23, 28, 65, 92, 17, 60, 13, 35, 27, 53, 75, 11, 68, 85, 63, 8, 14, 84, 42, 4, 22, 29, 10, 73, 88, 1, 57, 20, 19, 24, 7, 18, 21, 96, 32, 6, 34, 26]
[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 30, 36, 41, 82, 70, 38, 77, 39, 89, 78, 56, 83, 25, 45, 50, 100, 87, 3, 15, 93, 16, 81, 28, 17, 60, 37, 13, 74, 58, 53, 90, 75, 11, 68, 72, 63, 86, 14, 44, 80, 42, 29, 46, 88, 1, 57, 20, 7, 64, 18, 32, 33, 97, 69, 49, 26, 47, 2, 95]
[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 30, 36, 41, 82, 70, 38, 77, 39, 89, 78, 56, 83, 25, 45, 50, 100, 87, 3, 15, 93, 16, 81, 23, 65, 92, 37, 35, 74, 58, 27, 90, 72, 85, 86, 8, 44, 80, 84, 4, 22, 10, 46, 73, 19, 24, 64, 21, 96, 33, 97, 6, 69, 34, 49, 47, 2, 95, 12, 9]


In [14]:
transfee=[
    [0,1.95,1.4],
    [1.95,0,2.1],
    [1.4,2.1,0]
]

In [32]:
W=[]
W1=[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 61, 71, 30, 36, 41, 82, 38, 39, 78, 56, 83, 45, 50, 87, 3, 15, 81, 23, 28, 65, 92, 60, 37, 35, 74, 58, 53, 90, 75, 11, 72, 85, 8, 14, 44, 80, 46, 19, 21, 96, 33, 6, 26, 47, 2, 95, 12, 9]
W2=[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 5, 30, 36, 70, 77, 89, 83, 25, 100, 87, 3, 15, 93, 16, 23, 28, 17, 60, 13, 74, 27, 75, 11, 68, 85, 63, 86, 8, 14, 44, 4, 22, 73, 20, 24, 64, 18, 32, 97, 69, 34, 49]
W3=[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 41, 82, 70, 38, 77, 39, 89, 78, 56, 25, 45, 50, 100, 93, 16, 81, 65, 92, 17, 37, 13, 35, 58, 27, 53, 90, 68, 72, 63, 86, 80, 84, 42, 29, 10, 88, 1, 57, 7]


W11=[ str(t) for t in W1]
W12=[ str(t) for t in W2]
W13=[ str(t) for t in W3]

W.append(set(W11))
W.append(set(W12))
W.append(set(W13))

In [36]:
# 贪婪热销算法   #仓库大小60  采用可以转运的形式
# SKU_sort 为排好序的数组
# x个货品在三个仓库都有，y个货品在两个仓库里有，z个货品只有一个仓库有
# x+y+z=100
# 3*x+2*y+z=3*60
#  X=Z-20
#  y=80-2x
bestcost=9999999
bestW1=[]
bestW2=[]
bestW3=[]
bestx=0
c1=0
c2=0
for x in range(1,60):
    x=1
    # # 准备仓库
    # W1=[]
    # W2=[]
    # W3=[]

    # # x
    # for i in range(x):
    #     W1.append(SKU_sort[i]-1)
    #     W2.append(SKU_sort[i]-1)
    #     W3.append(SKU_sort[i]-1)

    # # print(W1)
    # # print(len(W1))


    # # y
    # if x<80-x:
    #     for i in range(x,80-x):

    #         item=SKU_sort[i]-1

    #         sum1=0
    #         if len(W1)<60:
    #             for j in W1:
    #                 sum1+=R[item][j]
    #         sum1=sum1/len(W1)
    #         #else :sum1=99999

    #         sum2=0
    #         if len(W2)<60:
    #             for j in W2:
    #                 sum2+=R[item][j]
    #         sum2=sum2/len(W2)
    #         #else :sum2=99999

    #         sum3=0
    #         if len(W3)<60:
    #             for j in W3:
    #                 sum3+=R[item][j]
    #         sum3=sum3/len(W3)
    #         #else :sum3=99999
            
    #         if sum1!=0 and sum2!=0 and sum3!=0:
    #             if min(sum1,sum2,sum3)==sum1:
    #                 W2.append(item)
    #                 W3.append(item)
    #             elif min(sum1,sum2,sum3)==sum2:
    #                 W1.append(item)
    #                 W3.append(item)
    #             elif min(sum1,sum2,sum3)==sum3:
    #                 W1.append(item)
    #                 W2.append(item)
            
    #         if sum1==0 or sum2==0 or sum3==0:
    #             if sum1!=0:
    #                 W1.append(item)
    #             if sum2!=0:
    #                 W2.append(item)
    #             if sum3!=0:
    #                 W3.append(item)
    #     # print(len(W1))
    #     # print(len(W2))

    # # z   
    # for i in range(80-x,100):

    #     item=SKU_sort[i]-1

    #     sum1=0
    #     if len(W1)<60:
    #         for j in W1:
    #             sum1+=R[item][j]
    #     sum1=sum1/len(W1)

    #     sum2=0
    #     if len(W2)<60:
    #         for j in W2:
    #             sum2+=R[item][j]
    #     sum2=sum2/len(W2)

    #     sum3=0
    #     if len(W3)<60:
    #         for j in W3:
    #             sum3+=R[item][j]
    #     sum3=sum3/len(W3)
            
    #     if max(sum1,sum2,sum3)==sum1 and sum1!=0:
    #                 W1.append(item)
    #     elif max(sum1,sum2,sum3)==sum2 and sum2!=0:
    #                 W2.append(item)
    #     elif max(sum1,sum2,sum3)==sum3 and sum3!=0:
    #                 W3.append(item)

    # # 序号从 0-99变为1—100
    # for i in range(len(W1)):
    #     W1[i]=str(W1[i]+1)
    # for i in range(len(W2)):
    #     W2[i]=str(W2[i]+1)
    # for i in range(len(W3)):
    #     W3[i]=str(W3[i]+1)
    # W=[]
    # W.append(set(W1))
    # W.append(set(W2))
    # W.append(set(W3))

    cost=[]  # 所有订单在一个x下的花费
    cm_1=0
    cm_2=0
    cm_3=0
    freq=np.zeros(13)
    for i in range(len(SKU_in_order)):
        #i=0
        if type(SKU_in_order[i])==str:
            k1=int(SKU_in_order[i])
            l1=[str(k1)]
            order_need=set(l1)
        else:
            order_need=set(SKU_in_order[i])

        pos=dis(pos_o[i])
        A=pos.index(min(pos))
        C=pos.index(max(pos))
        
        cost1=100
        cost2=100
        cost3=100
        cost4=100
        cost5=100
        cost6=100
        cost7=100
        cost8=100
        cost9=100
        cost10_1=100
        cost10_2=100
        cost11_1=100
        cost11_2=100
        cost12_1=100
        cost12_2=100


        for j in pos:
            if j<max(pos) and j >min(pos):
                B=pos.index(j)

        if min(pos)<5:
                costA=3
        else :
                costA=3+0.2*(min(pos)-5.0)

        if pos[B]<5:
                costB=3
        else :
                costB=3+0.2*(pos[B]-5.0)
        
        if max(pos)<5:
                costC=3
        else :
                costC=3+0.2*(max(pos)-5.0)

        # 仅需A就可以满足
        if order_need.issubset(W[A]):
            cost.append(costA)
            #freq[0]+=1
            continue
            # print("aaa")
        # 从其他两个仓库直接发出
        if order_need.issubset(W[B]):
            cost1=costB
        if order_need.issubset(W[C]):
            cost2=costC


        # AB
        if order_need.issubset(W[B]|W[A]):
            # trans_fee1=len(order_need.difference(W[A]))*transfee[A][B] #从A去的转运费
            # trans_fee2=len(order_need.difference(W[B]))*transfee[A][B] #从B去的转运费
            cost3=min(costA+costB,costA+len(order_need.difference(W[A]))*transfee[B][A],costB+len(order_need.difference(W[B]))*transfee[B][A])

            #continue
        # AC
        if order_need.issubset(W[C]|W[A]):
            trans_fee1=len(order_need.difference(W[A]))*transfee[A][C] #从A去的转运费
            trans_fee2=len(order_need.difference(W[C]))*transfee[A][C] #从C去的转运费
            cost4=min(costA+trans_fee1,costA+costC,costC+trans_fee2)
            #continue
        # BC
        if order_need.issubset(W[B]|W[C]):
            cost5=min(costB+costC,costB+len(order_need.difference(W[B]))*transfee[B][C],costC+len(order_need.difference(W[C]))*transfee[B][C])


        # 必须要三个仓库

        # 都直接去
        cost6=costA+costB+costC


        
        # A,B发货
        # 比较C->B和C->A的转运费
        # C->A  A,B发货
        if transfee[C][B]>transfee[C][A]:
            cost8=transfee[C][A]*len(order_need.difference(W[B]|W[A]))+costA+costB
        # C->B  A,B发货
        else:
            cost8=transfee[C][B]*len(order_need.difference(W[B]|W[A]))+costA+costB
        
        
        # AC发货
        # 比较B->A和B->C的转运费
        # C->客户 B->A
        if transfee[B][A]<=transfee[C][A]:
            cost9=costC+len(order_need.difference(W[C]|W[A]))*transfee[B][A]+costA
        # C->客户 B->C
        else:
            cost9=costC+len(order_need.difference(W[C]|W[A]))*transfee[C][B]+costA

        # BC发货
        # 比较A->B和A->C的转运费
        # A-B
        if transfee[A][B]<=transfee[A][C]:
            cost7=costC+len(order_need.difference(W[C]|W[B]))*transfee[B][A]+costB
        # C->客户 A->C
        else:
            cost7=costC+len(order_need.difference(W[C]|W[B]))*transfee[A][C]+costB    

            
        # B,C都转到A,统一由A去终点
        # 需要决定B,C转多少货品去A
        # 如果B去A的运费低，则C只运B没有的
        
        
        cost10_1=costA+transfee[C][A]*len(order_need.difference(W[A]|W[B]))+transfee[B][A]*(len(order_need.difference(W[A]))-len(order_need.difference(W[A]|W[B])))
        cost10_2=costA+transfee[B][A]*len(order_need.difference(W[A]|W[C]))+transfee[C][A]*(len(order_need.difference(W[A]))-len(order_need.difference(W[A]|W[C])))
        cost10=min(cost10_1,cost10_2)

        # C,A都转到B
        
        cost11_1=costB+transfee[C][B]*len(order_need.difference(W[A]|W[B]))+transfee[A][B]*(len(order_need.difference(W[B]))-len(order_need.difference(W[A]|W[B])))
        cost11_2=costB+transfee[A][B]*len(order_need.difference(W[C]|W[B]))+transfee[C][B]*(len(order_need.difference(W[B]))-len(order_need.difference(W[C]|W[B])))
        cost11=min(cost11_1,cost11_2)
        # A,B都转到C
        cost12_1=costC+transfee[A][C]*len(order_need.difference(W[C]|W[B]))+transfee[B][C]*(len(order_need.difference(W[C]))-len(order_need.difference(W[C]|W[B])))
        cost12_2=costC+transfee[B][C]*len(order_need.difference(W[C]|W[A]))+transfee[A][C]*(len(order_need.difference(W[C]))-len(order_need.difference(W[C]|W[A])))
        cost12=min(cost12_1,cost12_2)

        minn=min(cost1,cost2,cost3,cost4,cost5,cost6,cost7,cost8,cost9,cost10,cost11,cost12)
        if minn==cost6:
            cm_3+=1
        elif minn==cost7 or minn==cost8 or minn==cost9:
            cm_2+=1
        elif minn==cost3 and cost3==costA+costB:
            cm_2+=1
        elif minn==cost4 and cost3==costA+costC:
            cm_2+=1
        elif minn==cost5 and cost3==costB+costC:
            cm_2+=1
        else :
            cm_1+=1
        cost.append(minn)
    
    #计算对某一x，所有订单产生的花费
    # if cost<bestcost:
    #         bestcost=cost
    #         bestW1=W1
    #         bestW2=W2
    #         bestW3=W3
    #         c1=cm_1
    #         c2=cm_2
    #         c3=cm_3
    #         bestx=x
    break

In [38]:
print(sum(cost))
print(len(bestW1))
print(len(bestW2))
print(len(bestW3))
print(cm_1)
print(cm_2)
print(cm_3)
 


620975.2761733128
0
0
0
42524
0
0


In [39]:
W=[]
W1=[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 30, 36, 41, 82, 70, 38, 77, 39, 89, 78, 56, 83, 25, 45, 50, 100, 87, 3, 15, 93, 16, 23, 28, 65, 92, 17, 60, 13, 35, 27, 53, 75, 11, 68, 85, 63, 8, 14, 84, 42, 4, 22, 29, 10, 73, 88, 1, 57, 20, 19, 24, 7, 18, 21, 96, 32, 6, 34, 26]
W2=[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 30, 36, 41, 82, 70, 38, 77, 39, 89, 78, 56, 83, 25, 45, 50, 100, 87, 3, 15, 93, 16, 81, 28, 17, 60, 37, 13, 74, 58, 53, 90, 75, 11, 68, 72, 63, 86, 14, 44, 80, 42, 29, 46, 88, 1, 57, 20, 7, 64, 18, 32, 33, 97, 69, 49, 26, 47, 2, 95]
W3=[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 30, 36, 41, 82, 70, 38, 77, 39, 89, 78, 56, 83, 25, 45, 50, 100, 87, 3, 15, 93, 16, 81, 23, 65, 92, 37, 35, 74, 58, 27, 90, 72, 85, 86, 8, 44, 80, 84, 4, 22, 10, 46, 73, 19, 24, 64, 21, 96, 33, 97, 6, 69, 34, 49, 47, 2, 95, 12, 9]
W11=[ str(t) for t in W1]
W12=[ str(t) for t in W2]
W13=[ str(t) for t in W3]

W.append(set(W11))
W.append(set(W12))
W.append(set(W13))

In [50]:
# 贪婪热销算法   #仓库大小80  采用可以转运的形式
# SKU_sort 为排好序的数组
# x个货品在三个仓库都有，y个货品在两个仓库里有，z个货品只有一个仓库有
# x+y+z=100
# 3*x+2*y+z=3*80
#  X=Z+40
#  y=140-2x
bestcost=9999999
bestW1=[]
bestW2=[]
bestW3=[]
bestx=0
c1=0
c2=0
for x in range(40,80):
    x=42
    # # 准备仓库
    # W1=[]
    # W2=[]
    # W3=[]

    # # x
    # for i in range(x):
    #     W1.append(SKU_sort[i]-1)
    #     W2.append(SKU_sort[i]-1)
    #     W3.append(SKU_sort[i]-1)

    # # print(W1)
    # # print(len(W1))


    # # y
    # if x<140-x:
    #     for i in range(x,min(140-x,100)):

    #         item=SKU_sort[i]-1

    #         sum1=0
    #         if len(W1)<80:
    #             for j in W1:
    #                 sum1+=R[item][j]
    #         sum1=sum1/len(W1)
    #         #else :sum1=99999

    #         sum2=0
    #         if len(W2)<80:
    #             for j in W2:
    #                 sum2+=R[item][j]
    #         sum2=sum2/len(W2)
    #         #else :sum2=99999

    #         sum3=0
    #         if len(W3)<80:
    #             for j in W3:
    #                 sum3+=R[item][j]
    #         sum3=sum3/len(W3)
    #         #else :sum3=99999
            
    #         if sum1!=0 and sum2!=0 and sum3!=0:
    #             if min(sum1,sum2,sum3)==sum1:
    #                 W2.append(item)
    #                 W3.append(item)
    #             elif min(sum1,sum2,sum3)==sum2:
    #                 W1.append(item)
    #                 W3.append(item)
    #             elif min(sum1,sum2,sum3)==sum3:
    #                 W1.append(item)
    #                 W2.append(item)
            
    #         if sum1==0 or sum2==0 or sum3==0:
    #             if sum1!=0:
    #                 W1.append(item)
    #             if sum2!=0:
    #                 W2.append(item)
    #             if sum3!=0:
    #                 W3.append(item)
    #     # print(len(W1))
    #     # print(len(W2))

    # # z   
    # if 140-x<100:
    #     for i in range(140-x,100):
    #         item=SKU_sort[i]-1

    #         sum1=0
    #         if len(W1)<80:
    #             for j in W1:
    #                 sum1+=R[item][j]
    #         sum1=sum1/len(W1)

    #         sum2=0
    #         if len(W2)<80:
    #             for j in W2:
    #                 sum2+=R[item][j]
    #         sum2=sum2/len(W2)

    #         sum3=0
    #         if len(W3)<80:
    #             for j in W3:
    #                 sum3+=R[item][j]
    #         sum3=sum3/len(W3)
                    
    #         if max(sum1,sum2,sum3)==sum1 and sum1!=0:
    #             W1.append(item)
    #         elif max(sum1,sum2,sum3)==sum2 and sum2!=0:
    #             W2.append(item)
    #         elif max(sum1,sum2,sum3)==sum3 and sum3!=0:
    #             W3.append(item)

    # # 序号从 0-99变为1—100
    # for i in range(len(W1)):
    #     W1[i]=str(W1[i]+1)
    # for i in range(len(W2)):
    #     W2[i]=str(W2[i]+1)
    # for i in range(len(W3)):
    #     W3[i]=str(W3[i]+1)
    # W=[]
    # W.append(set(W1))
    # W.append(set(W2))
    # W.append(set(W3))

    # print(W1)
    # print(W2)
    # print(W3)
    cost=[]  # 所有订单在一个x下的花费
    cm_1=0
    cm_2=0
    cm_3=0
    for i in range(len(SKU_in_order)):
        #i=0
        if type(SKU_in_order[i])==str:
            k1=int(SKU_in_order[i])
            l1=[str(k1)]
            order_need=set(l1)
        else:
            order_need=set(SKU_in_order[i])

        pos=dis(pos_o[i])
        A=pos.index(min(pos))
        C=pos.index(max(pos))
        
        cost1=100
        cost2=100
        cost3=100
        cost4=100
        cost5=100
        cost6=100
        cost7=100
        cost8=100
        cost9=100
        cost10_1=100
        cost10_2=100
        cost11_1=100
        cost11_2=100
        cost12_1=100
        cost12_2=100


        for j in pos:
            if j<max(pos) and j >min(pos):
                B=pos.index(j)

        if min(pos)<5:
                costA=3
        else :
                costA=3+0.2*(min(pos)-5.0)

        if pos[B]<5:
                costB=3
        else :
                costB=3+0.2*(pos[B]-5.0)
        
        if max(pos)<5:
                costC=3
        else :
                costC=3+0.2*(max(pos)-5.0)

        # 仅需A就可以满足
        if order_need.issubset(W[A]):
            cost.append(costA)
            cm_1+=1
            #freq[0]+=1
            continue
            # print("aaa")
        # 从其他两个仓库直接发出
        if order_need.issubset(W[B]):
            cost1=costB
        if order_need.issubset(W[C]):
            cost2=costC


        # AB
        if order_need.issubset(W[B]|W[A]):
            # trans_fee1=len(order_need.difference(W[A]))*transfee[A][B] #从A去的转运费
            # trans_fee2=len(order_need.difference(W[B]))*transfee[A][B] #从B去的转运费
            cost3=min(costA+costB,costA+len(order_need.difference(W[A]))*transfee[B][A],costB+len(order_need.difference(W[B]))*transfee[B][A])

            #continue
        # AC
        if order_need.issubset(W[C]|W[A]):
            trans_fee1=len(order_need.difference(W[A]))*transfee[A][C] #从A去的转运费
            trans_fee2=len(order_need.difference(W[C]))*transfee[A][C] #从C去的转运费
            cost4=min(costA+trans_fee1,costA+costC,costC+trans_fee2)
            #continue
        # BC
        if order_need.issubset(W[B]|W[C]):
            cost5=min(costB+costC,costB+len(order_need.difference(W[B]))*transfee[B][C],costC+len(order_need.difference(W[C]))*transfee[B][C])


        # 必须要三个仓库

        # 都直接去
        cost6=costA+costB+costC


        
        # A,B发货
        # 比较C->B和C->A的转运费
        # C->A  A,B发货
        if transfee[C][B]>transfee[C][A]:
            cost8=transfee[C][A]*len(order_need.difference(W[B]|W[A]))+costA+costB
        # C->B  A,B发货
        else:
            cost8=transfee[C][B]*len(order_need.difference(W[B]|W[A]))+costA+costB
        
        
        # AC发货
        # 比较B->A和B->C的转运费
        # C->客户 B->A
        if transfee[B][A]<=transfee[C][A]:
            cost9=costC+len(order_need.difference(W[C]|W[A]))*transfee[B][A]+costA
        # C->客户 B->C
        else:
            cost9=costC+len(order_need.difference(W[C]|W[A]))*transfee[C][B]+costA

        # BC发货
        # 比较A->B和A->C的转运费
        # A-B
        if transfee[A][B]<=transfee[A][C]:
            cost7=costC+len(order_need.difference(W[C]|W[B]))*transfee[B][A]+costB
        # C->客户 A->C
        else:
            cost7=costC+len(order_need.difference(W[C]|W[B]))*transfee[A][C]+costB    

            
        # B,C都转到A,统一由A去终点
        # 需要决定B,C转多少货品去A
        # 如果B去A的运费低，则C只运B没有的
        
        
        cost10_1=costA+transfee[C][A]*len(order_need.difference(W[A]|W[B]))+transfee[B][A]*(len(order_need.difference(W[A]))-len(order_need.difference(W[A]|W[B])))
        cost10_2=costA+transfee[B][A]*len(order_need.difference(W[A]|W[C]))+transfee[C][A]*(len(order_need.difference(W[A]))-len(order_need.difference(W[A]|W[C])))
        cost10=min(cost10_1,cost10_2)

        # C,A都转到B
        
        cost11_1=costB+transfee[C][B]*len(order_need.difference(W[A]|W[B]))+transfee[A][B]*(len(order_need.difference(W[B]))-len(order_need.difference(W[A]|W[B])))
        cost11_2=costB+transfee[A][B]*len(order_need.difference(W[C]|W[B]))+transfee[C][B]*(len(order_need.difference(W[B]))-len(order_need.difference(W[C]|W[B])))
        cost11=min(cost11_1,cost11_2)
        # A,B都转到C
        cost12_1=costC+transfee[A][C]*len(order_need.difference(W[C]|W[B]))+transfee[B][C]*(len(order_need.difference(W[C]))-len(order_need.difference(W[C]|W[B])))
        cost12_2=costC+transfee[B][C]*len(order_need.difference(W[C]|W[A]))+transfee[A][C]*(len(order_need.difference(W[C]))-len(order_need.difference(W[C]|W[A])))
        cost12=min(cost12_1,cost12_2)

        minn=min(cost1,cost2,cost3,cost4,cost5,cost6,cost7,cost8,cost9,cost10,cost11,cost12)
        if minn==cost6:
            cm_3+=1
        elif minn==cost7 or minn==cost8 or minn==cost9:
            cm_2+=1
        elif minn==cost3 and cost3==costA+costB:
            cm_2+=1
        elif minn==cost4 and cost3==costA+costC:
            cm_2+=1
        elif minn==cost5 and cost3==costB+costC:
            cm_2+=1
        else :
            cm_1+=1
        cost.append(minn)
    
    #计算对某一x，所有订单产生的花费
    # if cost<bestcost:
    #         bestcost=cost
    #         bestW1=W1
    #         bestW2=W2
    #         bestW3=W3
    #         c1=cm_1
    #         c2=cm_2
    #         c3=cm_3
    #         bestx=x
    break

In [51]:
print(sum(cost))
print(cm_1)
print(cm_2)
print(cm_3)

526255.8007391231
50000
0
0


In [25]:
W=[]
W1=[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 30, 36, 41, 82, 70, 38, 77, 39, 89, 78, 56, 83, 25, 45, 50, 100, 87, 3, 15, 93, 16, 23, 28, 65, 92, 17, 60, 13, 35, 27, 53, 75, 11, 68, 85, 63, 8, 14, 84, 42, 4, 22, 29, 10, 73, 88, 1, 57, 20, 19, 24, 7, 18, 21, 96, 32, 6, 34, 26]
W2=[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 30, 36, 41, 82, 70, 38, 77, 39, 89, 78, 56, 83, 25, 45, 50, 100, 87, 3, 15, 93, 16, 81, 28, 17, 60, 37, 13, 74, 58, 53, 90, 75, 11, 68, 72, 63, 86, 14, 44, 80, 42, 29, 46, 88, 1, 57, 20, 7, 64, 18, 32, 33, 97, 69, 49, 26, 47, 2, 95]
W3=[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 30, 36, 41, 82, 70, 38, 77, 39, 89, 78, 56, 83, 25, 45, 50, 100, 87, 3, 15, 93, 16, 81, 23, 65, 92, 37, 35, 74, 58, 27, 90, 72, 85, 86, 8, 44, 80, 84, 4, 22, 10, 46, 73, 19, 24, 64, 21, 96, 33, 97, 6, 69, 34, 49, 47, 2, 95, 12, 9]
W11=[ str(t) for t in W1]
W12=[ str(t) for t in W2]
W13=[ str(t) for t in W3]
# W1={'14', '67', '49', '55', '94', '18', '40', '35', '65', '74', '21', '91', '28', '38', '56', '10', '17', '46', '5', '60', '15', '24', '47', '25', '45', '88', '53', '82', '97', '34', '6', '100', '1', '27', '48', '62', '69', '23', '11', '99', '57', '50', '68', '44', '20', '83', '3', '16', '12', '39', '43', '95', '64', '71', '84', '76', '73', '29', '31', '33', '75', '87', '9', '19', '78', '80', '13', '85', '58', '36'}
# W2={'77', '14', '67', '49', '55', '96', '94', '37', '65', '35', '41', '74', '91', '98', '38', '56', '10', '17', '46', '5', '60', '15', '54', '88', '8', '25', '45', '22', '53', '97', '34', '7', '6', '30', '27', '1', '48', '69', '70', '42', '66', '51', '2', '79', '99', '57', '50', '86', '26', '16', '12', '39', '43', '52', '92', '84', '76', '73', '29', '31', '33', '87', '89', '9', '59', '80', '85', '13', '93', '62'}
# W3={'77', '14', '67', '49', '55', '96', '32', '94', '40', '35', '41', '4', '28', '98', '56', '17', '46', '5', '60', '61', '81', '24', '47', '8', '25', '54', '88', '53', '82', '97', '34', '90', '100', '30', '1', '27', '69', '2', '11', '79', '99', '50', '68', '86', '44', '26', '20', '83', '3', '16', '12', '43', '52', '64', '92', '71', '72', '87', '73', '29', '31', '89', '59', '63', '78', '85', '13', '93', '58', '62'}
W.append(set(W11))
W.append(set(W12))
W.append(set(W13))


In [26]:
cost=[]

freq=np.zeros(13)
for i in range(len(SKU_in_order)):
    #i=0
    if type(SKU_in_order[i])==str:
        k1=int(SKU_in_order[i])
        l1=[str(k1)]
        order_need=set(l1)
    else:
        order_need=set(SKU_in_order[i])

    pos=dis(pos_o[i])
    A=pos.index(min(pos))
    C=pos.index(max(pos))
    
    cost1=100
    cost2=100
    cost3=100
    cost4=100
    cost5=100
    cost6=100
    cost7=100
    cost8=100
    cost9=100
    cost10_1=100
    cost10_2=100
    cost11_1=100
    cost11_2=100
    cost12_1=100
    cost12_2=100


    for j in pos:
        if j<max(pos) and j >min(pos):
            B=pos.index(j)

    if min(pos)<5:
            costA=3
    else :
            costA=3+0.2*(min(pos)-5.0)

    if pos[B]<5:
            costB=3
    else :
            costB=3+0.2*(pos[B]-5.0)
    
    if max(pos)<5:
            costC=3
    else :
            costC=3+0.2*(max(pos)-5.0)

    # 仅需A就可以满足
    if order_need.issubset(W[A]):
        cost.append(costA)
        freq[0]+=1
        continue
        # print("aaa")
    # 从其他两个仓库直接发出
    if order_need.issubset(W[B]):
        cost1=costB
    if order_need.issubset(W[C]):
        cost2=costC


    # AB
    if order_need.issubset(W[B]|W[A]):
        # trans_fee1=len(order_need.difference(W[A]))*transfee[A][B] #从A去的转运费
        # trans_fee2=len(order_need.difference(W[B]))*transfee[A][B] #从B去的转运费
        cost3=min(costA+costB,costA+len(order_need.difference(W[A]))*transfee[B][A],costB+len(order_need.difference(W[B]))*transfee[B][A])

        if cost3==costA+costB:
            print("a")
        #continue
    # AC
    if order_need.issubset(W[C]|W[A]):
        trans_fee1=len(order_need.difference(W[A]))*transfee[A][C] #从A去的转运费
        trans_fee2=len(order_need.difference(W[C]))*transfee[A][C] #从C去的转运费
        cost4=min(costA+trans_fee1,costA+costC,costC+trans_fee2)
        #continue
    # BC
    if order_need.issubset(W[B]|W[C]):
        cost5=min(costB+costC,costB+len(order_need.difference(W[B]))*transfee[B][C],costC+len(order_need.difference(W[C]))*transfee[B][C])


    # 必须要三个仓库

    # 都直接去
    cost6=costA+costB+costC


    
    # A,B发货
    # 比较C->B和C->A的转运费
    # C->A  A,B发货
    if transfee[C][B]>transfee[C][A]:
        cost8=transfee[C][A]*len(order_need.difference(W[B]|W[A]))+costA+costB
    # C->B  A,B发货
    else:
        cost8=transfee[C][B]*len(order_need.difference(W[B]|W[A]))+costA+costB
    
    
    # AC发货
    # 比较B->A和B->C的转运费
    # C->客户 B->A
    if transfee[B][A]<=transfee[C][A]:
        cost9=costC+len(order_need.difference(W[C]|W[A]))*transfee[B][A]+costA
    # C->客户 B->C
    else:
        cost9=costC+len(order_need.difference(W[C]|W[A]))*transfee[C][B]+costA

    # BC发货
    # 比较A->B和A->C的转运费
    # A-B
    if transfee[A][B]<=transfee[A][C]:
        cost7=costC+len(order_need.difference(W[C]|W[B]))*transfee[B][A]+costB
    # C->客户 A->C
    else:
        cost7=costC+len(order_need.difference(W[C]|W[B]))*transfee[A][C]+costB    

        
    # B,C都转到A,统一由A去终点
    # 需要决定B,C转多少货品去A
    # 如果B去A的运费低，则C只运B没有的
    
    
    cost10_1=costA+transfee[C][A]*len(order_need.difference(W[A]|W[B]))+transfee[B][A]*(len(order_need.difference(W[A]))-len(order_need.difference(W[A]|W[B])))
    cost10_2=costA+transfee[B][A]*len(order_need.difference(W[A]|W[C]))+transfee[C][A]*(len(order_need.difference(W[A]))-len(order_need.difference(W[A]|W[C])))
    cost10=min(cost10_1,cost10_2)

    # C,A都转到B
    
    cost11_1=costB+transfee[C][B]*len(order_need.difference(W[A]|W[B]))+transfee[A][B]*(len(order_need.difference(W[B]))-len(order_need.difference(W[A]|W[B])))
    cost11_2=costB+transfee[A][B]*len(order_need.difference(W[C]|W[B]))+transfee[C][B]*(len(order_need.difference(W[B]))-len(order_need.difference(W[C]|W[B])))
    cost11=min(cost11_1,cost11_2)
    # A,B都转到C
    cost12_1=costC+transfee[A][C]*len(order_need.difference(W[C]|W[B]))+transfee[B][C]*(len(order_need.difference(W[C]))-len(order_need.difference(W[C]|W[B])))
    cost12_2=costC+transfee[B][C]*len(order_need.difference(W[C]|W[A]))+transfee[A][C]*(len(order_need.difference(W[C]))-len(order_need.difference(W[C]|W[A])))
    cost12=min(cost12_1,cost12_2)

    minn=min(cost1,cost2,cost3,cost4,cost5,cost6,cost7,cost8,cost9,cost10,cost11,cost12)
    if minn==cost1:
        freq[1]+=1    
        #print("1")
    elif minn==cost2:
        freq[2]+=1
        #print("2")
    elif minn==cost3:
        freq[3]+=1
        #print("3")
    elif minn==cost4:
        freq[4]+=1
        #print("4")
    elif minn==cost5:
        freq[5]+=1
        #print("5")
    elif minn==cost6:
        freq[6]+=1
        #print("6")
    elif minn==cost7:
        freq[7]+=1
        #print("7")
    elif minn==cost8:
        freq[8]+=1
        #print("8")
    elif minn==cost9:
        freq[9]+=1
        #print("9")
    elif minn==cost10:
        freq[10]+=1
        #print("10")
    elif minn==cost11:
        freq[11]+=1
        #print("11")
    elif minn==cost12:
        freq[12]+=1
        #print("12")
    cost.append(minn)
    #break

In [27]:
print(sum(cost))

526255.8007391231


In [22]:
W=[]
W1=[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 30, 36, 41, 82, 70, 38, 77, 39, 89, 78, 56, 83, 25, 45, 50, 100, 87, 3, 15, 93, 16, 23, 28, 65, 92, 17, 60, 13, 35, 27, 53, 75, 11, 68, 85, 63, 8, 14, 84, 42, 4, 22, 29, 10, 73, 88, 1, 57, 20, 19, 24, 7, 18, 21, 96, 32, 6, 34, 26]
W2=[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 30, 36, 41, 82, 70, 38, 77, 39, 89, 78, 56, 83, 25, 45, 50, 100, 87, 3, 15, 93, 16, 81, 28, 17, 60, 37, 13, 74, 58, 53, 90, 75, 11, 68, 72, 63, 86, 14, 44, 80, 42, 29, 46, 88, 1, 57, 20, 7, 64, 18, 32, 33, 97, 69, 49, 26, 47, 2, 95]
W3=[99, 66, 91, 62, 43, 52, 59, 31, 76, 54, 51, 67, 98, 94, 40, 55, 79, 48, 61, 71, 5, 30, 36, 41, 82, 70, 38, 77, 39, 89, 78, 56, 83, 25, 45, 50, 100, 87, 3, 15, 93, 16, 81, 23, 65, 92, 37, 35, 74, 58, 27, 90, 72, 85, 86, 8, 44, 80, 84, 4, 22, 10, 46, 73, 19, 24, 64, 21, 96, 33, 97, 6, 69, 34, 49, 47, 2, 95, 12, 9]

W1=set(W1)
W2=set(W2)
W3=set(W3)
W.append(W1)
W.append(W2)
W.append(W3)

print(W1)
print(W[0])

{1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 38, 39, 40, 41, 42, 43, 45, 48, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 65, 66, 67, 68, 70, 71, 73, 75, 76, 77, 78, 79, 82, 83, 84, 85, 87, 88, 89, 91, 92, 93, 94, 96, 98, 99, 100}
{1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 38, 39, 40, 41, 42, 43, 45, 48, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 65, 66, 67, 68, 70, 71, 73, 75, 76, 77, 78, 79, 82, 83, 84, 85, 87, 88, 89, 91, 92, 93, 94, 96, 98, 99, 100}
